In [1]:
# ====================================================
# 01 - Data Cleaning (dual outputs)
# Goal:
#   - The output will be an ML-ready version and a version for EDA which does not apply all filters and keep all columns that would be interesting for EDA but not necessary for ML.
# ====================================================

In [ ]:
# --- 1) Setup ---
import os, re, ast
import pandas as pd
import numpy as np

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_DIR = os.path.join(PROJECT_ROOT, "data")
PROC_DIR = os.path.join(DATA_DIR, "processed")
os.makedirs(PROC_DIR, exist_ok=True)

FILTERED_PATH = os.path.join(PROC_DIR, "filtered_project_data_with_descriptions.csv")
OUT_ML_READY  = os.path.join(PROC_DIR, "cleaned_project_data.csv")
OUT_EDA_FULL  = os.path.join(PROC_DIR, "cleaned_project_data_eda.csv")

# Optional: Google Drive file ID for filtered input (fallback download)
FILTERED_DRIVE_FILE_ID = os.environ.get("FILTERED_DRIVE_FILE_ID", "18FQxIKGF32Qi-RZtLFVg7HHcFRsFMBOF")

print("Project root:", PROJECT_ROOT)
print("Processed dir:", PROC_DIR)
print("Filtered path:", FILTERED_PATH)
print("ML-ready out :", OUT_ML_READY)
print("EDA full out :", OUT_EDA_FULL)

# Load filtered input (local first, then Drive)
if not os.path.exists(FILTERED_PATH):
    print("Filtered file not found locally. Downloading from Google Drive…")
    try:
        import gdown
    except ImportError:
        raise RuntimeError("gdown is not installed. Run `pip install gdown` or use Colab.")
    gdown.download(id=FILTERED_DRIVE_FILE_ID, output=FILTERED_PATH, quiet=False)

if not os.path.exists(FILTERED_PATH):
    raise FileNotFoundError("filtered_project_data_with_descriptions.csv not found.")

raw = pd.read_csv(FILTERED_PATH, encoding="utf-8")
print(f"Loaded filtered dataset: shape={raw.shape}")
raw.head(3)

Project root: /Users/timschnelzer/Developer/tender-matching-horizon-europe
Processed dir: /Users/timschnelzer/Developer/tender-matching-horizon-europe/data/processed
Filtered path: /Users/timschnelzer/Developer/tender-matching-horizon-europe/data/processed/filtered_project_data_with_descriptions.csv
ML-ready out : /Users/timschnelzer/Developer/tender-matching-horizon-europe/data/processed/cleaned_project_data.csv
EDA full out : /Users/timschnelzer/Developer/tender-matching-horizon-europe/data/processed/cleaned_project_data_eda.csv
Loaded filtered dataset: shape=(17797, 21)


,id,acronym,status,title,startDate,endDate,totalCost,ecMaxContribution,legalBasis,topics,...,frameworkProgramme,masterCall,subCall,fundingScheme,objective,contentUpdateDate,rcn,grantDoi,keywords,topics_description
0,101234994,OPTIMALMINE,SIGNED,OPTIMALMINE: slope optimal design for a paradi...,2025-09-01,2029-08-31,0,1072140,HORIZON.1.2,HORIZON-MSCA-2024-SE-01-01,...,HORIZON,HORIZON-MSCA-2024-SE-01,HORIZON-MSCA-2024-SE-01,HORIZON-TMA-MSCA-SE,The European Union is currently addressing cha...,2025-07-25 11:08:05,274682,10.3030/101234994,"mine optimisation, rock slope engineering, o...",Expected Outcome:\nProject results are expecte...
1,101232577,HSAFE,SIGNED,Innovative high-sensitivity avalanche field-ef...,2025-09-01,2029-08-31,0,1618230,HORIZON.1.2,HORIZON-MSCA-2024-SE-01-01,...,HORIZON,HORIZON-MSCA-2024-SE-01,HORIZON-MSCA-2024-SE-01,HORIZON-TMA-MSCA-SE,"The focus of the HSAFE project, which aligns w...",2025-07-25 11:08:05,274696,10.3030/101232577,"Field-effect transistor-based biosensors, Canc...",Expected Outcome:\nProject results are expecte...
2,101236527,DRU,SIGNED,Democratic Roles of Universities (DRU): Practi...,2026-02-01,2030-01-31,0,1593180,HORIZON.1.2,HORIZON-MSCA-2024-SE-01-01,...,HORIZON,HORIZON-MSCA-2024-SE-01,HORIZON-MSCA-2024-SE-01,HORIZON-TMA-MSCA-SE,DRU’s objective is to find new ways that unive...,2025-07-25 11:08:04,274676,10.3030/101236527,"Universities, Citizen science, Civic engagement",Expected Outcome:\nProject results are expecte...


In [ ]:
# --- 2) Cleaning helpers ---

def _clean_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    return re.sub(r"\s+", " ", text).strip()


def _clean_title(text: str) -> str:
    if not isinstance(text, str):
        return ""
    text = re.sub(r"[^\w\s]", "", text)
    return re.sub(r"\s+", " ", text).strip().lower()


def _clean_topics_description(text: str) -> str:
    if not isinstance(text, str):
        return ""
    # remove headings like "Expected outcome:" / "Scope:" (anywhere)
    text = re.sub(r"(?i)\b(expected outcome:|scope:)\b", "", text)
    # remove UI artifacts anywhere
    text = re.sub(r"(?i)\bshow\s*less\b", "", text)
    text = re.sub(r"(?i)\bshow\s*more\b", "", text)
    # remove bracketed refs like [1], [2]
    text = re.sub(r"\[[0-9]+\]", "", text)
    # collapse whitespace
    return re.sub(r"\s+", " ", text).strip()


def _clean_keywords(keywords) -> list:
    if isinstance(keywords, list):
        return [str(k).strip().lower() for k in keywords if str(k).strip()]
    if isinstance(keywords, str):
        s = keywords.strip()
        if s.startswith("[") and s.endswith("]"):
            try:
                return _clean_keywords(ast.literal_eval(s))
            except Exception:
                pass
        return [k.strip().lower() for k in s.split(",") if k.strip()]
    return []


def _extract_year(s: str) -> int:
    if not isinstance(s, str):
        return -1
    m = re.search(r"\b(2021|2022|2023|2024|2025)\b", s)
    return int(m.group(1)) if m else -1


In [ ]:
# Shared enrichment (non-destructive)
df = raw.copy()

In [ ]:
# --- 3) Enrichments ---

# Standardize names
df = df.rename(columns={"objective": "project_objective", "title": "project_title"})

# Derived/cleaned text fields
df["project_title_clean"]      = df.get("project_title", "").apply(_clean_title)
df["project_objective_clean"]  = df.get("project_objective", "").apply(_clean_text)
df["topics_description_clean"] = df.get("topics_description", "").apply(_clean_topics_description)

# Keywords (list + string)
df["keywords_clean"] = df.get("keywords", "").apply(_clean_keywords)
df["keywords_str"]   = df["keywords_clean"].apply(lambda xs: " ".join(xs) if isinstance(xs, list) else "")

# Composite text features
# Making sure for topic_text that "Show less" (remainder from web scraping) is removed
df["topic_text"] = (
    df.get("topics", "").astype(str).str.strip() + " " +
    df["topics_description_clean"].astype(str).str.strip()
).str.replace(r"(?i)\bshow\s*less\b", "", regex=True) \
 .str.replace(r"(?i)\bshow\s*more\b", "", regex=True) \
 .str.replace(r"\s+", " ", regex=True).str.strip()
df["project_text_simple"]  = (df["project_title_clean"] + " " + df["project_objective_clean"]).str.strip()
df["project_text_keywords"] = (df["project_text_simple"] + " keywords: " + df["keywords_str"]).str.strip()

# Year + one-hot
if "year" not in df.columns:
    df["year"] = df.get("topics", "").astype(str).apply(_extract_year)
for y in [-1, 2021, 2022, 2023, 2024, 2025]:
    col = f"year_{y}"
    if col not in df.columns:
        df[col] = (df["year"] == y).astype(int)

print("Enriched columns prepared for both outputs.")

Enriched columns prepared for both outputs.


In [ ]:
# --- 4) ML-READY OUTPUT ---
# Apply ONLY here the generic scheme filtering and column restriction.

GENERIC_PREFIXES = ("ERC", "HORIZON-MSCA", "HORIZON-WIDERA", "HORIZON-EIC", "HORIZON-EIE")
mask_keep = ~df["topics"].astype(str).str.startswith(GENERIC_PREFIXES)
df_ml_ready = df.loc[mask_keep].reset_index(drop=True)

print(f"Rows after filtering (ML-ready): {len(df_ml_ready)} / {len(df)}")
print(f"Unique topics (ML-ready): {df_ml_ready['topics'].nunique()}")

cols = [
    "id", "topics",
    "project_text_simple", "project_text_keywords",
    "topic_text", "keywords_clean",
    "year_-1","year_2021","year_2022","year_2023","year_2024","year_2025",
]
cols = [c for c in cols if c in df_ml_ready.columns]
df_ml_ready_out = df_ml_ready.reindex(columns=cols)

# Save ML-ready
df_ml_ready_out.to_csv(OUT_ML_READY, index=False, encoding="utf-8")
print(f"Saved ML-ready cleaned CSV (strict) to: {OUT_ML_READY}  shape={df_ml_ready_out.shape}")

Rows after filtering (ML-ready): 4429 / 17797
Unique topics (ML-ready): 1873
Saved ML-ready cleaned CSV (strict) to: /Users/timschnelzer/Developer/tender-matching-horizon-europe/data/processed/cleaned_project_data.csv  shape=(4429, 12)


In [7]:
# Dropping irrelevant columns for EDA output
columns_to_drop = [
    'acronym',
    'legalBasis',
    'ecSignatureDate',
    'frameworkProgramme',
    'masterCall',
    'subCall',
    'fundingScheme',
    'project_objective',
    'contentUpdateDate',
    'rcn',
    'grantDoi',
    'keywords',
    'topics_description_clean',
    'project_title_clean',
    'project_objective_clean',
    'keywords_clean',
    'topics_description'
]

df = df.drop(columns=columns_to_drop, errors='ignore')
display(df.head())

,id,status,project_title,startDate,endDate,totalCost,ecMaxContribution,topics,keywords_str,topic_text,project_text_simple,project_text_keywords,year,year_-1,year_2021,year_2022,year_2023,year_2024,year_2025
0,101234994,SIGNED,OPTIMALMINE: slope optimal design for a paradi...,2025-09-01,2029-08-31,0,1072140,HORIZON-MSCA-2024-SE-01-01,mine optimisation rock slope engineering optim...,HORIZON-MSCA-2024-SE-01-01 Expected Outcome: P...,optimalmine slope optimal design for a paradig...,optimalmine slope optimal design for a paradig...,2024,0,0,0,0,1,0
1,101232577,SIGNED,Innovative high-sensitivity avalanche field-ef...,2025-09-01,2029-08-31,0,1618230,HORIZON-MSCA-2024-SE-01-01,field-effect transistor-based biosensors cance...,HORIZON-MSCA-2024-SE-01-01 Expected Outcome: P...,innovative highsensitivity avalanche fieldeffe...,innovative highsensitivity avalanche fieldeffe...,2024,0,0,0,0,1,0
2,101236527,SIGNED,Democratic Roles of Universities (DRU): Practi...,2026-02-01,2030-01-31,0,1593180,HORIZON-MSCA-2024-SE-01-01,universities citizen science civic engagement,HORIZON-MSCA-2024-SE-01-01 Expected Outcome: P...,democratic roles of universities dru practicin...,democratic roles of universities dru practicin...,2024,0,0,0,0,1,0
3,101236483,SIGNED,3D topological states in solid and soft ferroe...,2026-02-01,2030-01-31,0,1117230,HORIZON-MSCA-2024-SE-01-01,ferroelectrics liquid crystals topological states,HORIZON-MSCA-2024-SE-01-01 Expected Outcome: P...,3d topological states in solid and soft ferroe...,3d topological states in solid and soft ferroe...,2024,0,0,0,0,1,0
4,101235387,SIGNED,IoT Supported Electronic Geotextiles for Susta...,2026-01-01,2029-12-31,0,1683360,HORIZON-MSCA-2024-SE-01-01,,HORIZON-MSCA-2024-SE-01-01 Expected Outcome: P...,iot supported electronic geotextiles for susta...,iot supported electronic geotextiles for susta...,2024,0,0,0,0,1,0


In [ ]:
# --- 5) EDA FULL OUTPUT (no row filtering; keep ALL columns) ---

df_eda = df.copy()

# Save EDA CSV
df_eda.to_csv(OUT_EDA_FULL, index=False, encoding="utf-8")
print(f"Saved EDA FULL CSV (no filtering) to: {OUT_EDA_FULL}  shape={df_eda.shape}")

# Quick previews
print("\nML-ready preview:")
print(df_ml_ready_out.head(3))
print("\nEDA full preview (columns count =", len(df_eda.columns), "):")
print(df_eda.head(3))
print("\nColumns (EDA):", list(df_eda.columns))

Saved EDA FULL CSV (no filtering) to: /Users/timschnelzer/Developer/tender-matching-horizon-europe/data/processed/cleaned_project_data_eda.csv  shape=(17797, 19)

ML-ready preview:
          id                                         topics  \
0  101225859                      HORIZON-CL3-2024-CS-01-02   
1  101226137                     HORIZON-CL3-2024-DRS-01-02   
2  101181402  HORIZON-CL6-2024-ZEROPOLLUTION-02-2-two-stage   

                                 project_text_simple  \
0  functional composition of post quantum cryptos...   
1  towards enhanced coordination of disaster risk...   
2  olinwaste smart sustainable biorefining of oli...   

                               project_text_keywords  \
0  functional composition of post quantum cryptos...   
1  towards enhanced coordination of disaster risk...   
2  olinwaste smart sustainable biorefining of oli...   

                                          topic_text  \
0  HORIZON-CL3-2024-CS-01-02 Expected Outcome: Pr...   
1  H